In [1]:
from pymongo import MongoClient

from pprint import pprint

# (a) Pour se connecter à MongoDB via pymongo, ajoutez l'authentification aux lignes de codes suivantes puis lancez-les
client = MongoClient(
    host="127.0.0.1",
    port = 27017,
    username = "datascientest",
    password = "dst123"
)

In [2]:
# Afficher la liste des bases de données disponibles.
print(client.list_database_names())

['admin', 'config', 'local', 'sample']


In [3]:
# (c) Afficher la liste des collections disponibles dans cette base de données.
sample = client["sample"]
print(sample.list_collection_names())

['rand', 'books', 'weather', 'zips', 'grades', 'cie']


In [4]:
# (d) Afficher un des documents de cette collection.
from pprint import pprint

pprint(sample["books"].find_one())

{'_id': 1,
 'author_1': 'W. Frank Ableson',
 'author_2': 'Charlie Collins',
 'author_3': 'Robi Sen',
 'authors': ['W. Frank Ableson', 'Charlie Collins', 'Robi Sen'],
 'categories': ['Open Source', 'Mobile'],
 'isbn': '1933988673',
 'longDescription': 'Android is an open source mobile phone platform based on '
                    'the Linux operating system and developed by the Open '
                    'Handset Alliance, a consortium of over 30 hardware, '
                    'software and telecom companies that focus on open '
                    'standards for mobile devices. Led by search giant, '
                    'Google, Android is designed to deliver a better and more '
                    'open and cost effective mobile experience.    Unlocking '
                    "Android: A Developer's Guide provides concise, hands-on "
                    'instruction for the Android operating system and '
                    'development tools. This book teaches important '
           

In [ ]:
#pprint(list(sample["books"].find({},{"pageCount":1}))) // Pour test

In [5]:
# (e) Afficher le nombre de documents dans cette collection*
pprint(sample["books"].count_documents({}))


431


# _Exploration de la base_

In [20]:
# (a) Afficher le nombre de livres avec plus de 400 pages, affichez ensuite le nombre de livres ayant plus de 400 pages ET qui sont publiés.

# Nombre de libres avec plus de 400 pages
print("Nombre de livres :",
    sample["books"].count_documents(
        {"pageCount": {"$gte": 400}},
    )
)

print("Nombre de livres publié de plus de 400 pages :",
    sample["books"].count_documents(
        {
            "$and": [
                {"pageCount": {"$gte": 400}},
                {"status": "PUBLISH"},
            ]
        }
    )
)

Nombre de livres : 168
Nombre de livres publié de plus de 400 pages : 166


In [29]:
# (b) Afficher le nombre de livres ayant le mot-clé Android dans leur description (brève ou longue).
print("Nombre de livres ayant le mot-clé Android :",
    sample["books"].count_documents(
        {
            "$or": [
                {"shortDescription":{"$regex": "Android"}},
                {"longDescription":{"$regex": "Android"}}
            ]
        }
    )
)

Nombre de livres ayant le mot-clé Android : 4


In [ ]:
#pprint(list(sample["books"].find({},{"categories":1}).limit(5))) 
# (c) Chaque document possède un attribut categories qui est une liste. 
# Vous devez grouper tous les documents en un à l'aide de l'opérateur $group. 
# Puis, à l'aide de l'opérateur $addToSet, créez 2 set à partir des catégories contenus dans la liste categories selon leur index 0 ou 1. 
# Pour cibler, les catégories utilisez l'opérateur $arrayElemAt.


pprint(list(sample["books"].aggregate(
            [
                {"$group": {"_id":None, 
                        "categorie_0": {
                            "$addToSet": {
                                "$arrayElemAt":["$categories",0]
                            }
                        },
                        "categorie_1":{
                            "$addToSet": {
                                "$arrayElemAt":["$categories",1]
                            }
                        }}
                
                },
                # Affiche uniquement 20 lignes de chaque catégorie afin de vérifier que l'agrégat retourne bien les deux catégories.
                {"$project" :{ "_id":0, "categorie_0":{"$slice":["$categorie_0",20]}, "categorie_1":{"$slice":["$categorie_1",20]}}},
            ]
        )
    )
)
    

[{'categorie_0': ['Open Source',
                  'Algorithmic Art',
                  'Microsoft/.NET',
                  'internet',
                  'Theory',
                  'Programming',
                  'Computer Graphics',
                  'Next Generation Databases',
                  'Miscellaneous',
                  'Mobile Technology',
                  'Python',
                  'Software Engineering',
                  'XML',
                  'Networking',
                  'PHP',
                  'Web Development',
                  'Microsoft .NET',
                  'PowerBuilder',
                  'In Action',
                  'Perl'],
  'categorie_1': ['Java',
                  'Theory',
                  'Perl',
                  '',
                  'P',
                  'Networking',
                  'Mobile',
                  'Software Engineering',
                  '.NET',
                  'Open Source',
                  'Internet',
          

In [35]:
# (d) Afficher le nombre de livres qui contiennent des noms de langages suivant dans leur description longue : 
# Python, Java, C++, Scala. 
# On pourra s'appuyer sur des expressions régulières et une condition or.
print("Nombre de livres ayant des noms de langages dans leur description :",
    sample["books"].count_documents(
        {"longDescription":{"$regex": "Python|Java|C++|Scala"}}
    )
)


Nombre de livres ayant des noms de langages dans leur description : 195


In [ ]:
# (e) Afficher diverses informations statistiques sur notre bases de données : 
# nombre maximal, minimal, et moyen de pages par catégorie. 
# On utilisera une pipeline d'aggregation, le mot clef $group, ainsi que les accumulateurs appropriés. 
# N'oubliez pas d'aller voir "$unwind" pour ce problème.
pprint(list(sample["books"].aggregate(
            [
                {"$unwind": "$categories"},
                {"$group": {"_id":"$categories",
                           "nb_max":{"$max" :"$pageCount"}, 
                           "nm_min":{"$min":"$pageCount"},
                           "avg":{"$avg":"$pageCount"}
                           }
                },
            ]
        )
    )
)


In [ ]:
# (f) Via une pipeline d'aggrégation, Créer de nouvelles variables en extrayant info 
# depuis l'attribut dates : année, mois, jour. On rajoutera une condition pour filtrer seulement les livres publiés après 2009. 
# N'affichez que les 20 premiers.

pipeline = [
            {"$project": {
                  "year": {"$year": "$publishedDate"},
                  "month": {"$month": "$publishedDate"},
                  "day": {"$dayOfMonth": "$publishedDate"}
                  }
            },
            {"$match": {
                  "year": {"$gte": 2009}
                  }},
            {"$limit": 20}
]

pprint(list(sample["books"].aggregate(pipeline)))

                

In [ ]:
# (g) À partir de la liste des auteurs, créez de nouveaux attributs (author_1, author_2 ... author_n). 
# Observez le comportement de "$arrayElemAt". N'affichez que les 20 premiers dans l'ordre chronologique.

# Récupération du nombre maximum d'auteurs dans tous les documents
nb_auteurs = list(sample["books"].aggregate(
        [
            {"$project": {
                "nb_auteur" : {"$size":"$authors"}
                }
            },
            {
            "$group": {"_id":0,"nb_autheur":{"$max":"$nb_auteur"}}
            },
        ]
    )
)
print("Nombre maximum d'auteurs par document :", nb_auteurs)

#création d'un pipeline pour créer les nouvelles colonnes
# Question : Comment rendre la création des attributs dynamiques ?
pipeline = [
    {"$set": {
        "author_1": {"$arrayElemAt":["$authors",0]},
        "author_2": {"$arrayElemAt":["$authors",1]},
        "author_3": {"$arrayElemAt":["$authors",2]},
        "author_4": {"$arrayElemAt":["$authors",3]},
        "author_5": {"$arrayElemAt":["$authors",4]},
        "author_6": {"$arrayElemAt":["$authors",5]},
        "author_7": {"$arrayElemAt":["$authors",6]},
        "author_8": {"$arrayElemAt":["$authors",7]}
    }},
    {"$limit": 20},
    {"$sort":{"publishedDate":1}},
    {"$merge":"books"}, # applique la création sur la collection
]

sample["books"].aggregate(pipeline)

#Check
pprint(sample["books"].find_one({"_id":2}))


Nombre maximum d'auteur par document : [{'_id': 0, 'nb_autheur': 8}]
{'_id': 2,
 'author_1': 'W. Frank Ableson',
 'author_2': 'Robi Sen',
 'authors': ['W. Frank Ableson', 'Robi Sen'],
 'categories': ['Java'],
 'isbn': '1935182722',
 'longDescription': 'When it comes to mobile apps, Android can do almost '
                    'anything   and with this book, so can you! Android runs '
                    'on mobile devices ranging from smart phones to tablets to '
                    "countless special-purpose gadgets. It's the broadest "
                    'mobile platform available.    Android in Action, Second '
                    'Edition is a comprehensive tutorial for Android '
                    'developers. Taking you far beyond "Hello Android," this '
                    "fast-paced book puts you in the driver's seat as you "
                    'learn important architectural concepts and implementation '
                    "strategies. You'll master the SDK, build WebKit ap

In [ ]:
#(h) En s'inspirant de la requête précédente, créer une colonne contenant le nom du premier auteur, 
# puis agréger selon cette colonne pour obtenir le nombre d'articles pour chaque premier auteur. 
# Afficher le nombre de publications pour les 10 premiers auteurs les plus prolifiques. 
# On pourra utiliser un pipeline d'agrégation avec les mots-clefs $group, $sort, $limit.

#Creation d'un pipeline d'agrégation
pipeline = [
    {"$set": {
        "author_1": {"$arrayElemAt":["$authors",0]},
    }},
    {"$group": {
        "_id":"$author_1", "nb_artcile":{"$sum":1}
    }},
    {"$sort":{"nb_artcile":-1}},
    {"$limit":10}
]

pprint(list(sample["books"].aggregate(pipeline)))

[{'_id': None, 'nb_artcile': 37},
 {'_id': 'Vikram Goyal', 'nb_artcile': 12},
 {'_id': 'Don Jones', 'nb_artcile': 6},
 {'_id': 'Christian Bauer', 'nb_artcile': 4},
 {'_id': 'Craig Walls', 'nb_artcile': 4},
 {'_id': 'Erik Hatcher', 'nb_artcile': 3},
 {'_id': 'David A. Black', 'nb_artcile': 3},
 {'_id': 'Jon Skeet', 'nb_artcile': 3},
 {'_id': 'W. Frank Ableson', 'nb_artcile': 3},
 {'_id': 'Tim Hatton', 'nb_artcile': 3}]


In [ ]:
#(i) [OPTIONNEL] Afficher la distribution du nombre d'auteurs : 
# Commencer par créer une nouvelle colonne avec le nombre d'auteurs (taille de la liste de l'attribut authors ), 
# puis agrégez sur cette colonne avec l'accumulateur $count ou $sum.

#Creation d'un pipeline d'agrégation
pipeline = [
    {"$set": {
        "nb_author": {"$size":"$authors"},
    }},
    {"$group": {
        "_id":"$nb_author", "dist":{"$sum":1}
    }},
    # Question : Comment définir l'ordre d'affichage ? Dans ce cas, la distribution s'affiche avant le nb auteur.
    {"$project": {"_id":0, "nb_author":"$_id","dist":1}}, 
    {"$sort":{"dist":-1}}
]

pprint(list(sample["books"].aggregate(pipeline)))

[{'dist': 206, 'nb_author': 1},
 {'dist': 105, 'nb_author': 2},
 {'dist': 47, 'nb_author': 4},
 {'dist': 37, 'nb_author': 0},
 {'dist': 16, 'nb_author': 5},
 {'dist': 9, 'nb_author': 3},
 {'dist': 6, 'nb_author': 6},
 {'dist': 3, 'nb_author': 8},
 {'dist': 2, 'nb_author': 7}]


In [50]:
#(j) [OPTIONNEL] Afficher l'occurence de chaque auteur selon son index dans l'attribut "authors".
# Un même auteur peut avoir plusieurs index. 
# N'affichez pas les auteurs vides, sortez par ordre d'occurence décroissant avec une limite de 20. 
# Utilisez "$unwind" pour séparer les auteurs et "$project" pour supprimer les auteurs absents.

pprint(list(sample["books"].aggregate(
            [
                {"$unwind": "$authors"},
                {"$group": {"_id":"$authors","nb_occurence":{"$sum":1}}},
                {"$project": {"_id":0, "author":"$_id", "nb_occurence":1}},
                {"$match":{"author": {"$ne":""}}},
                {"$sort":{"nb_occurence":-1}},
                {"$limit":10}
            ]
        )
    )
)

[{'author': 'Vikram Goyal', 'nb_occurence': 12},
 {'author': 'Richard Siddaway', 'nb_occurence': 6},
 {'author': 'Don Jones', 'nb_occurence': 6},
 {'author': 'Jon Skeet', 'nb_occurence': 5},
 {'author': 'Yehuda Katz', 'nb_occurence': 5},
 {'author': 'Gavin King', 'nb_occurence': 5},
 {'author': 'Christian Bauer', 'nb_occurence': 5},
 {'author': 'Craig Walls', 'nb_occurence': 4},
 {'author': 'Kalen Delaney', 'nb_occurence': 4},
 {'author': 'Greg Low', 'nb_occurence': 4}]
